$\newcommand{\set}[2]{\big\{#1\,\ {\large:}\ \,#2\big\}}
\newcommand{\eps}{\varepsilon}
\newcommand{\abs}[1]{\left\vert#1\right\vert}
\newcommand{\ceil}[1]{\left\lceil#1\right\rceil}
\newcommand{\floor}[1]{\left\lfloor#1\right\rfloor}
$
# O método da bissecção

## $ \S 1 $ Introdução

O método da bissecção é um método geral para determinação de zeros de uma função  real _contínua_ $ f $. Seu único requerimento é que tenhamos previamente identificado um intervalo $ [a, b] $ dentro do domínio de $ f $ onde ela troca de sinal. Suas principais vantagens são:
* _Confiabilidade_: ele sempre converge a um zero, satisfeita a hipótese acima.
* _Simplicidade_: não é necessário impor condições adicionais sobre a função nem calcular sua derivada.
* _Rapidez_: para localizar um zero com erro menor que $ \eps > 0 $, ele necessita de um número de passos proporcional ao _logaritmo_ de $ \frac{b-a}{\eps} $. Em particular, este número não depende da função.
* _Robustez_: pequenos erros de arredondamento (comparados à precisão desejada) não impactam seu resultado.

Por outro lado, o método não pode ser empregado para se localizar um zero que não pode ser encaixotado (como o da função $ x \mapsto x^2 $). Além disto, em muitos casos é possível utilizar métodos cuja convergência é ainda mais rápida. Se queremos obter um zero de uma única função, esta diferença de desempenho não é relevante, porém a situação muda se o algoritmo precisa ser executado bilhões ou trilhões de vezes.

## $ \S 2 $ Descrição do método da bissecção

Como acima, sejam $ f $ uma função real _contínua_ e $ [a, b] $ um intervalo dentro do domínio de $ f $ com
$$
f(a)f(b) < 0.
$$
No **método da bissecção**, começamos tomando $ m $ como o **ponto médio** de $ a $ e $ b $,
$$
m = \frac{a+b}{2}\,,
$$
e avaliando $ f $ aí. Há três possibilidades:
* Se $ f(m)f(a) = 0 $, então $ m $ é um zero de $ f $ e podemos terminar.
* Se $ f(m)f(a) < 0 $, então $ f $ troca de sinal em $ [a, m] $.
* Se $ f(m)f(a) > 0 $, então $ f $ troca de sinal em $ [m, b] $.

Nos dois últimos casos, podemos restringir nossa busca a um intervalo cujo comprimento é a *metade* do anterior. O Teorema do Valor Intermediário garante que este subintervalo ainda contém um zero, portanto podemos repetir o processo. Bissecções sucessivas eventualmente nos levarão a um zero ou a um encaixotamento de um zero por um subintervalo de comprimento tão pequeno quanto desejado. Quando isto acontecer, retornamos o ponto médio deste subintervalo como estimativa para o zero.

Por exemplo, dez bissecções cortam o comprimento do intervalo original por um fator de $ 2^{10} > 1\,000 $. Mais geralmente, temos o seguinte resultado.

**Teorema 1 (análise do método da bissecção):** _Pelo método da bissecção, o número de iterações necessário para se localizar um zero dentro do intervalo inicial $ [a, b] $ com erro menor que $ \eps > 0 $ é dado por:_
\begin{equation*}\label{E:1}
\boxed{\ceil{\lg\bigg(\frac{b-a}{\eps}\bigg)} \quad \text{onde $ \lg = \log_2 $}} \tag{1}
\end{equation*}

**Prova:** Começando com o intervalo $ [a,b] $ original (primeiro passo), sabemos que em cada iteração há um zero dentro do intervalo atual. Após o $ n $-ésimo passo, o comprimento do intervalo original terá sido reduzido pelo fator de $ 2^{n-1} $. Mas como o resultado do método é o ponto médio do intervalo atual, o que importa é a *metade* do seu comprimento. Logo $ n $ deve ser grande o suficiente de modo que
$$
    \frac{b-a}{2^n} < \eps, \quad \text{ou equivalentemente,} \quad 
    n > \lg \left( \frac{b-a}{\eps} \right).
$$
O menor inteiro que satisfaz esta desigualdade é o teto do valor à direita, como em \eqref{E:1}. <div style="text-align: right">$ \blacksquare $ </div>

📝 Informalmente, o resultado acima implica que para cada dígito adicional de precisão, precisamos de $ \lg 10 \approx 3.32 $ iterações a mais. Este desempenho deve ser comparado com o da busca incremental, que dependia _linearmente_ de $ \frac{b-a}{\eps} $; portanto para melhorar a precisão desta por um dígito decimal, precisamos _multiplicar_ o número de iterações por $ 10 $.

## $ \S 3 $ Critérios de parada e observações

Os critérios de parada que podemos utilizar são essencialmente os mesmos em todos os métodos para localização de zeros:

1. O comprimento do intervalo atual é menor que um $ \eps > 0 $ pré-escolhido.
2. O número de iterações excede uma cota prefixada.
3. O módulo do valor da função no ponto médio $ m $ atual é menor que um $ \delta > 0 $ escolhido previamente.
4. O módulo do valor da função no ponto médio $ m $, atual relativamente aos valores originais $ f(a) $ e $ f(b) $, é menor que um $ \delta > 0 $ escolhido previamente, i.e.:
$$
    \frac{\abs{f(m)}}{\min\left\{\abs{f(a)}\,,\,\abs{f(b)}\right\}} < \delta.
$$

Na implementação abaixo, o procedimento termina assim que algum dos critérios 1 ou 2 for satisfeito.

⚠️ Para que possamos sequer aplicar o método da bissecção a uma função, antes é necessário ter obtido um intervalo onde ela troca de sinal. Este requerimento preliminar geralmente pode ser satisfeito com a inspeção do gráfico da função ou com uma busca incremental (conforme explicado no caderno anterior).

⚠️ Se $ f $ tiver uma descontinuidade do tipo *pólo* de ordem ímpar num ponto (como a função tangente em $ \frac{\pi}{2} $), então o método da bissecção poderá confundi-lo com um zero; veja a figura abaixo.

![Gráfico da função tangente](fig_2-3_grafico_tan.png "Title")

## $ \S 3 $ Implementação do método da bissecção

In [2]:
def bisseccao(f, a, b, eps):
    """
    Utiliza o método da bissecção para localizar um zero de uma função.
    Entradas:
        * A função real contínua f.
        * As extremidades a e b de um intervalo onde f troca de sinal.
        * A distância máxima tolerada eps do resultado a um zero.
    Saída:
        * Um ponto a distância menor que eps de um zero de f.
    """
    from numpy import sign, log2, ceil
    
    
    iteracoes = 1                         # Contador do número de iterações.
    f_a = f(a)                            # Gravando o valor de f em a.
    f_b = f(b)                            # Gravando o valor de f em b.
    if eps <= 0:
        raise ValueError("A tolerância deve ser positiva!")
    if f_a == 0:
        return a
    elif f_b == 0:
        return b
    elif sign(f_a) == sign(f_b):        # Verifique que [a, b] contém um zero.
        raise ValueError("A função assume valores de mesmo sinal "
                         "nas extremidades dadas!")
    
    while 0.5 * (b - a) >= eps:
        m = 0.5 * (a + b)                 # Ponto médio do intervalo anterior.
        f_m = f(m)                        # Gravando o valor de f em m.
        if f_m == 0:
            print("Encontrado um zero exato!")
            return m
        elif sign(f_a) != sign(f_m):     # O intervalo [a, m] contém um zero.
            b = m                         # Tome o novo b como sendo m.
        else:                             # O intervalo [m, b] contém um zero.
            a = m                         # Tome o novo a como sendo m.
            f_a = f_m
        iteracoes += 1
        
    m = 0.5 * (a + b)
    print(f"Encontrado um zero aproximado:\n{m:12.7f}")
    print(f"após {iteracoes} iterações, com erro de no máximo {0.5 * (b - a)}.")
    print(f"O valor da função neste ponto é:\n{f(m):12.7f}")
    return m

## $ \S 4 $ Problemas

**Problema 1:**

(a) Estime $ \sqrt[3]{2} $ com $ 5 $ dígitos decimais de precisão usando o método da bissecção. *Dica:* Considere a função $ f(x) = x^3 - 2 $.

(b) Use o Teorema 1 para calcular o número de iterações necessárias.

*Solução:*

**Problema 2:** Seja $ y = \cot x $ a função cotangente. Recorde que 
$$
\cot\left(-\frac{\pi}{4}\right) = -1 \quad \text{e} \quad \cot\left(\frac{\pi}{3}\right) = \frac{1}{\sqrt{3}}
$$
Aplique o método da bissecção a $ f = \cot $ no intervalo $ \left[-\frac{\pi}{4}\,,\,\frac{\pi}{3}\right] $ e explique o resultado.

*Solução:*

**Problema 3:**

(a) Encontre a menor raiz real positiva da equação
$$
x^3 - 3.45x^2 - 5.72 x + 6.31 = 0
$$
com uma precisão melhor que $ 10^{-3} $ usando o método da bissecção.

(b) Considerando o intervalo original que você utilizou, quantos passos são necessários?

*Solução:*

**Problema 4:** Obtenha uma estimativa para $ \pi $, correta até a sexta casa decimal, utilizando o método da bissecção para aproximar o primeiro zero positivo da função seno. 

*Solução:*

**Problema 5:**

(a) Mostre que a equação $ xe^x = 1 $ possui uma única raiz em $ \mathbb{R} $. *Dica:* Calcule a derivada da função $ f(x) = xe^x - 1 $.

(b) Mostre que esta raiz se encontra no intervalo $ [0.5, 1] $.

(c) Estime esta raiz com erro máximo de $ 10^{-4} $, usando o método da bissecção.

(d) Quantos passos são necessários?

*Solução:*

**Problema 6:** Resolva o exercício anterior para a equação $ x^2 + \ln x = 0 $.

*Solução:*

**Problema 7:** Encontre uma raiz de cada uma das equações abaixo com erro $ < 10^{-3} $. *Dica:* Antes de aplicar o método da bissecção, é necessário encaixotar um zero. Para isto, utilize a análise gráfica ou a busca incremental. Os procedimentos correspondentes estão na última seção deste caderno.

(a) $ x \ln x = 1 $.

(b) $ \cos x = x^2 $.

(c) $ x^5 - 3x^4 - 6x^3 + 4x^2 + 5x - 3 = 0 $.

(d) $ \tan x = x + 2 e^x $.

(e) $ \cos x \cosh x = 1 $, onde por definição $ \cosh x = \frac{e^x + e^{-x}}{2} $.

*Solução:*

## $ \S 5 $ Procedimentos auxiliares dos cadernos passados

In [ ]:
def busca_incremental(f, a, b, h):
    """
    Começando com x_1 = a e x_2 = a + h e com incrementos
    de h, retorna o primeiro par de pontos consecutivos onde
    f assume sinais opostos.
    """
    from numpy import sign
    
    # Inicializando:
    x_0 = a
    x_1 = a + h
    f_0 = f(x_0)
    f_1 = f(x_1)
    
    while sign(f_0) == sign(f_1):
        if x_1 > b:
            return None, None
        x_0, f_0 = x_1, f_1
        x_1 += h
        f_1 = f(x_1)
        
    return x_0, x_1

In [ ]:
def plota_funcoes(a, b, N, *fs):
    """
    Entradas:
        * Extremidades a < b de um intervalo.
        * Número N de pontos na amostra dos valores de x.
        * Um número qualquer (>= 1) de funções definidas em [a, b].
    Exibe num mesmo diagrama o gráfico das funções e retorna None.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    
    x = np.linspace(a, b, N)                    # Amostra de valores de x
    for i, f in enumerate(fs):
        plt.plot(x, f(x), label=f'função {i}')  # Plotar dados: (x, y, etiqueta).
    plt.xlabel('Eixo-x')
    plt.ylabel('Eixo-y')
    plt.grid(True)
    plt.title("Gráfico simples")
    plt.legend()
    
    return None